In [1]:
from typing import Dict
import flwr as fl
import multiprocessing as mp
from flower_helpers import (create_model, get_weights, test, load_data)
from config import (NUM_ROUNDS, MODEL_NAME, NUM_CLASSES, 
                    PRE_TRAINED, TRAIN_SIZE, VAL_PORTION, 
                    TEST_SIZE, BATCH_SIZE, LEARNING_RATE, 
                    EPOCHS, FRAC_FIT, FRAC_EVAL, MIN_FIT,
                    MIN_EVAL, MIN_AVAIL, FIT_CONFIG_FN,
                    NUM_CLIENTS, CLIENT_RESOURCES)
from client import FlowerClient

In [2]:
# mp.set_start_method("spawn")

In [3]:
trainloaders, valloaders, testloader = load_data()

Found cached dataset cifar10 (C:/Users/Jean/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at C:\Users\Jean\.cache\huggingface\datasets\cifar10\plain_text\1.0.0\447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4\cache-16b9e105e7ead8c5.arrow
Loading cached shuffled indices for dataset at C:\Users\Jean\.cache\huggingface\datasets\cifar10\plain_text\1.0.0\447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4\cache-6d58bc2a635b7b42.arrow


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
net = create_model()
init_weights = get_weights(net)
MODEL_CONFIG = net.config
# Convert the weights (np.ndarray) to parameters (bytes)
init_param = fl.common.ndarrays_to_parameters(init_weights)
# del the net as we don't need it anymore
del net

In [5]:
# server side evaluation function
def evaluate(server_round: int, params: fl.common.NDArrays,
             config: Dict[str, fl.common.Scalar]):
    data_size, metrics = test(MODEL_CONFIG, params, testloader)
    # changing the name of the metric to avoid confusion
    metrics['test_loss'] = metrics.pop('loss')
    metrics['test_accuracy'] = metrics.pop('accuracy')
    return metrics['test_loss'], metrics

def weighted_average_eval(metrics):
    weighted_train_loss = 0
    weighted_train_accuracy = 0
    for c in metrics: # c is a tuple (num_examples, metrics) for each client
        weighted_train_loss += c[0] * c[1]['val_loss']
        weighted_train_accuracy += c[0] * c[1]['val_accuracy']
    
    aggregated_metrics = {'val_loss': weighted_train_loss / sum([c[0] for c in metrics]),
            'val_accuracy': weighted_train_accuracy / sum([c[0] for c in metrics])}
    print('\t',aggregated_metrics)
    return aggregated_metrics

def weighted_average_fit(metrics):
    # print(metrics)
    weighted_train_loss = 0
    weighted_train_accuracy = 0
    for c in metrics: # c is a tuple (num_examples, metrics) for each client
        # metrics for each epoch is included, we only need the last one
        weighted_train_loss += c[0] * c[1]['train_loss'][-1]
        weighted_train_accuracy += c[0] * c[1]['train_accuracy'][-1]
    
    aggregated_metrics = {'train_loss': weighted_train_loss / sum([c[0] for c in metrics]),
            'train_accuracy': weighted_train_accuracy / sum([c[0] for c in metrics])}
    print('\t',aggregated_metrics)
    return aggregated_metrics

In [6]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRAC_FIT,
    fraction_evaluate=FRAC_EVAL,
    min_fit_clients=MIN_FIT,
    min_evaluate_clients=MIN_EVAL,
    min_available_clients=MIN_AVAIL,
    
    fit_metrics_aggregation_fn=weighted_average_fit,
    evaluate_metrics_aggregation_fn=weighted_average_eval,
    evaluate_fn=evaluate,
    on_fit_config_fn=FIT_CONFIG_FN,
    
    initial_parameters=init_param,
)

In [7]:
fl.simulation.start_simulation(
    client_fn=lambda cid: FlowerClient(MODEL_CONFIG, trainloaders[int(cid)], valloaders[int(cid)]),
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
    client_resources=CLIENT_RESOURCES,
)

INFO flwr 2023-04-09 12:29:47,264 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-04-09 12:29:49,293	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-09 12:29:50,589 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 6864548659.0, 'CPU': 24.0, 'GPU': 1.0, 'memory': 13729097319.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-04-09 12:29:50,590 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-09 12:29:50,590 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-04-09 12:29:50,591 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-09 12:30:08,908 | server.py:91 | initial parameters (loss, other metrics): 17.05419921875, {'test_loss': 17.05419921875, 'test_accuracy': 0.11}
INFO flwr 2023-04-09 12:30:08,909 | server.py:101 | FL starting
DEBUG flwr 2023-04-09 12:30:08,909 | server.py:215 | fit_roun

	 {'train_loss': 1.3714876174926758, 'train_accuracy': 0.3333333333333333}


INFO flwr 2023-04-09 12:35:26,763 | server.py:116 | fit progress: (1, 5.994919776916504, {'test_loss': 5.994919776916504, 'test_accuracy': 0.45}, 317.8836947999953)
DEBUG flwr 2023-04-09 12:35:26,764 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-04-09 12:35:42,261 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-04-09 12:35:42,261 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 5)


	 {'val_loss': 1.5944551229476929, 'val_accuracy': 0.4}


DEBUG flwr 2023-04-09 12:40:37,463 | server.py:229 | fit_round 2 received 2 results and 0 failures


...

DEBUG flwr 2023-04-09 13:26:03,001 | server.py:229 | fit_round 10 received 2 results and 0 failures


	 {'train_loss': 0.7861122786998749, 'train_accuracy': 0.8444444444444444}


INFO flwr 2023-04-09 13:26:21,031 | server.py:116 | fit progress: (10, 2.039668560028076, {'test_loss': 2.039668560028076, 'test_accuracy': 0.78}, 3372.337421300006)
DEBUG flwr 2023-04-09 13:26:21,032 | server.py:165 | evaluate_round 10: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-04-09 13:26:36,286 | server.py:179 | evaluate_round 10 received 2 results and 0 failures
INFO flwr 2023-04-09 13:26:36,287 | server.py:144 | FL finished in 3387.593078299993
INFO flwr 2023-04-09 13:26:36,302 | app.py:202 | app_fit: losses_distributed [(1, 1.5944551229476929), (2, 1.33826744556427), (3, 0.9811164140701294), (4, 0.6709871590137482), (5, 0.8732505738735199), (6, 0.42904285341501236), (7, 0.5959099233150482), (8, 0.27168411761522293), (9, 0.621816024184227), (10, 0.6743447482585907)]
INFO flwr 2023-04-09 13:26:36,302 | app.py:203 | app_fit: metrics_distributed {'val_loss': [(1, 1.5944551229476929), (2, 1.33826744556427), (3, 0.9811164140701294), (4, 0.6709871590137482), (5, 0.8732505738

	 {'val_loss': 0.6743447482585907, 'val_accuracy': 0.8}


History (loss, distributed):
	round 1: 1.5944551229476929
	round 2: 1.33826744556427
	round 3: 0.9811164140701294
	round 4: 0.6709871590137482
	round 5: 0.8732505738735199
	round 6: 0.42904285341501236
	round 7: 0.5959099233150482
	round 8: 0.27168411761522293
	round 9: 0.621816024184227
	round 10: 0.6743447482585907
History (loss, centralized):
	round 0: 17.05419921875
	round 1: 5.994919776916504
	round 2: 4.977465629577637
	round 3: 3.8078718185424805
	round 4: 3.0592617988586426
	round 5: 2.419541358947754
	round 6: 2.62664794921875
	round 7: 2.8388242721557617
	round 8: 2.3509411811828613
	round 9: 2.275913715362549
	round 10: 2.039668560028076
History (metrics, distributed):
{'val_loss': [(1, 1.5944551229476929), (2, 1.33826744556427), (3, 0.9811164140701294), (4, 0.6709871590137482), (5, 0.8732505738735199), (6, 0.42904285341501236), (7, 0.5959099233150482), (8, 0.27168411761522293), (9, 0.621816024184227), (10, 0.6743447482585907)], 'val_accuracy': [(1, 0.4), (2, 0.575), (3, 0.6